# Ejercicio 1

In [2]:
'''Función que nos genera una población inicial de 2k individuos y 
   recibe como parámetros: La 'k' y el hipercubo al que pertenecen 
   los vectores (o sea, la dimensión 'n' y el intervalo [a,b] al que
   pertenecen cada una de las entradas de los vectores).'''

import numpy as np

def Generar_Poblacion(k, n, intervalo):
    
   a, b = intervalo

   # Se genera una matriz de tamaño (2k, n) con valorios aleatorios
   # uniformes (o sea, una población inicial de 2k individuos donde 
   # cada uno vive en Rn):
   poblacionMatriz = np.random.uniform(a,b, size=(2*k, n))

   # Para facilitarnos la vida, convertimos a la matriz en una lista
   # de listas:
   poblacionLista = poblacionMatriz.tolist()

   return poblacionLista


In [3]:
'''Definimos una función de aptitud que recibe como parámetros una
   función de prueba y una población de 2k individuos y retorna una
   lista con las probabilidades de selección (p_i) de todos los 
   individuos de la población. Además de que nos da al mejor individuo
   en la población.'''

def Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion):
    # Generamos una lista con las evaluaciones en la función objetivo
    evaluaciones = [funcion(individuo) for individuo in poblacion]

    #print("Evaluaciones:", [float(eval) for eval in evaluaciones], "\n")

    # Generamos una lista de las aptitudes de los individuos en la 
    # población.
    aptitudes = [np.exp(-evaluacion) for evaluacion in evaluaciones]

    #print("Aptitudes:", [float(apt) for apt in aptitudes], "\n")

    # Encontramos el índice del mejor individuo (menor evaluación)
    mejorIndice = np.argmax(aptitudes)
    masApto = poblacion[mejorIndice]

    #print("El individuo más apto fue:", masApto, "\n")

    # Obtenemos el 'tamaño del pastel':
    sumaAptitudes = sum(aptitudes)

    # Calculamos las probabilidades de selección (p_i) para todos los
    # individuos:
    probabilidades = [(aptitud/sumaAptitudes) for aptitud in aptitudes]

    #print("Probabilidades de selección:", [float(pi) for pi in probabilidades])


    return probabilidades, masApto


In [5]:
'''Definimos una función que recibe la lista de probabilidades 
   de selección y nos retorna la que corresponde a la de las 
   probabilidades acumulativas.'''

def Probas_Acumulativas(probabilidades):

    # Calculamos las probabilidades acumulativas (q_i) para todos los
    # individuos:
    probasAcumulativas = []
    sumaAcumulada = 0

    for proba in probabilidades:
        sumaAcumulada += proba
        probasAcumulativas.append(sumaAcumulada)

    #return probasAcumulativas
    return probasAcumulativas


In [6]:
'''Estas funciones son las que nos ayudan a generar a los padres mediante
   el uso de la ruleta.'''
   

'''La primera es para seleccionar padres (uno) y recibe como parámetro 
   la lista de las probabilidades acumulativas.
   
   Nos retornará el índice en la lista que le corresponde al padre 
   elegido con esa regla.'''

import random
import math

def Seleccionar_Padre_Ruleta(probasAcumuladas):
   r = random.random() # Generamos un número aleatorio entre 0 y 1

   tol=1e-9
   for i, probAcumulada in enumerate(probasAcumuladas):
      if r<= probAcumulada or math.isclose(r, probAcumulada, abs_tol=tol):
         return i # Índice del individuo seleccionado.


'''Le segunda es para generar la lista de padres elegidos para 
   reproducirse. Necesitamos 2k nuevos individuos, así que se van a 
   elegir k parejas (cada par de padres produce dos hijos).
   La función recibe como parámetros: la lista de individuos, las
   probabilidades aculativas y un máximo de intentos para intentar evitar
   la generación de clones (en este caso, vamos a considerar un máximo
   de 100 para cada elección de un segundo padre, pero tenemos la 
   apertura de modificarlo después).'''

def Generar_Parejas(individuos, probasAcumuladas, maxIntentos=100):
    parejas = []
    clonesGenerados = 0  # Contador de clones generados
    numIndividuos = len(individuos)

    if numIndividuos < 2:
        print("Error: No hay suficientes individuos para generar parejas.")
        return parejas, clonesGenerados

    # Como len(individuos)=2k, le pedimos que elija a len(individuos)/2
    # parejas de padres (que nos generarán 2k hijos).
    parejasNecesarias = int(numIndividuos/2)
    #print(f"Se necesitan {parejasNecesarias} parejas de padres.")

    for _ in range(parejasNecesarias):
        padre1_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

        intentos = 0
        while True:
            padre2_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

            if padre1_idx != padre2_idx:
                break  # Padres diferentes, salir del ciclo
            
            intentos += 1
            if intentos >= maxIntentos:  # Después de varios intentos, permitir clones
                padre2_idx = padre1_idx
                clonesGenerados += 2 # Cada pareja produce dos hijos
                #print(f"Se ha generado un clon tras {intentos} intentos.")
                break  # Salir del ciclo permitiendo el clon

        # Agregamos la pareja solo si hay padres válidos
        if padre1_idx is not None and padre2_idx is not None:
            parejas.append((individuos[padre1_idx], individuos[padre2_idx]))
        else:
            print("Error: No se pudieron seleccionar padres válidos.")

    return parejas, clonesGenerados


In [7]:
'''Funciones auxiliares para la codificación en binario.'''

def codifica_real(x, n_bit, intervalo):
    """
    Codifica un número real en el intervalo [a,b] utilizando nBit bits y una partición uniforme en [a, b].

    Parámetros:
    x: número a real a codificar.
    nBit: número de bits a utilizar.
    a: Extremo izquierdo del intervalo.
    b: Extremo derecho del intervalo.

    Retorno:
    Arreglo binario que representa al número real x con nBit bits en el intervalo [a, b].
    """
    
    a, b = intervalo
    
    # Calcula la precisión de la representación.
    precision = (b - a) / (2 ** n_bit)

    # Asegura que el número esté dentro del rango de la partición.
    x = max(a, min(b, x))

    # Calcula el índice del número en la partición.
    index = int((x - a) / precision)

    # Codifica el índice a binario usando nuestro codigo para codificar naturales.
    if index < 0 or index >= (1 << n_bit):
        raise ValueError(f"Índice fuera del rango representable con {n_bit} bits.")

    x_binario = [0] * n_bit
    for i in range(n_bit - 1, -1, -1):
        x_binario[i] = index & 1
        index >>= 1

    # Devuelve el arreglo binario que representa al número real 'x'.
    return x_binario

def codifica_vector(vector_reales, n_bit, intervalo):
    """
    Codifica un vector de números reales en un vector de vectores binarios utilizando nBit bits.

    Parámetros:
    vector_reales: Arreglo de números reales a codificar
    dim_x: Diimensión del vector vector_reales
    nBit: Número de bits a utilizar para las entradas de nuestro arreglo.
    a: Extremo izquierdo del intervalo.
    b: Extremo derecho del intervalo.

    Retorno:
    Arreglo de arreglos arreglos binarios, donde cada subarreglo representa un número real codificado en binario
    """
    
    dim_x = len(vector_reales)

    # Inicializa un arreglo vacío para almacenar los vectores binarios.
    vector_binario = []

    # Itera sobre cada entrada de nuestro vector de reales.
    for i in range(dim_x):
        numero = vector_reales[i]

        # Codifica cada número utilizando la función codifica
        binario = codifica_real(numero, n_bit, intervalo)

        # Añade el resultado codificado al vector principal
        vector_binario.append(binario)

    # Devuelve un arreglo de arreglos binarios.
    return vector_binario


#print(f"El vector {[1.4,1.2,3,1.5674]} en binario con {10} bits es:\n")
#print(codifica_vector([1.4,1.2,3,1.5674],10,(1,5)))


In [8]:
'''Codificación de los padres en vectores binarios del estilo
   [[0,1,1,1,...], ..., [1, 1, 1, 0,...]]'''

def Padres_Binarios(parejas, nBits, intervalo):
   parejasBinarias = []

   for padre1, padre2 in parejas:

      padre1Bin = codifica_vector(padre1, nBits, intervalo)
      padre2Bin = codifica_vector(padre2, nBits, intervalo)

      parejasBinarias.append((padre1Bin, padre2Bin))
   
   return parejasBinarias


In [9]:
'''Función que realiza la cruza de n puntos. Recibe como parámetros: la
   lista de los padres (en arreglos binarios), el número de cortes que 
   se quieren hacer por cromosoma y el número de bits usados en la 
   codificación binaria. Retorna una lista de 2k hijos (como arreglos 
   binarios).'''

def Cruzar_N_Puntos(padresBinarios, nCortes, nBits):
    
    hijos = [] # Lista donde se guardará a los 2k hijos
    
    # Los `nCortes` son un parámetro recibido e idealmente es el que se
    # usará, en caso de que `nCortes` sea demasido grande, se hará el 
    # mayor número de cortes posible, o sea `nBits - 1`
    n = min(nCortes, (nBits - 1))
    
    for padre1, padre2 in padresBinarios:
        
        # Los hijos se van a crear cruzando cada cromosoma de los padres. 
        # Recordemos que cada pareja produce dos hijos.
        hijo1 = []
        hijo2 = []
        
        # Cruzamos cada cromosoma individualmente
        for cromo1, cromo2 in zip(padre1, padre2): # `zip(padre1, padre2)` toma los cromosomas 
                                                   # correspondientes de ambos padres al mismo 
                                                   # tiempo, es decir, empareja los cromosomas 
                                                   # que están en las mismas posiciones de las 
                                                   # listas `padre1` y `padre2`.
            
            # Determinar los puntos de cruce
            puntosCruce = sorted(random.sample(range(1, len(cromo1)), n))

            #print(puntosCruce) # En la primera iteración, si tenemos [1, 5]
                               # esto quiere decir que en el primer cromosoma
                               # de la pareja de padres, se van a hacer cruces
                               # en tres pedazos determinados por los dos cortes
                               # en las posiciones 1 y 5.
            
            # Inicializamos cromosomas vacíos para los hijos
            hijo1Cromo = []
            hijo2Cromo = []
            
            ultimoPunto = 0 # Punto en el que vamos para los cruces, inicia en 0.
            
            switch = False  # Esta bandera indica si debemos cambiar de padre
            
            # Alternamos entre segmentos de los padres en los puntos de cruza
            for punto in (puntosCruce + [len(cromo1)]): # Añadimos el valor len(cromo1) 
                                                        # al final de la lista de puntos de 
                                                        # cruza para asegurarnos de que el 
                                                        # último segmento después del último 
                                                        # punto de corte también se incluya. 
                                                        # De esta manera, recorremos todo el 
                                                        # cromosoma.
                
                # Seleccionamos los segmentos entre el último punto y el punto actual
                if switch:
                    hijo1Cromo += cromo2[ultimoPunto:punto] # El cromosoma del hijo1 recibe 
                                                            # parte del cromosoma del segundo
                                                            # padre.
                    hijo2Cromo += cromo1[ultimoPunto:punto] # El cromosoma del hijo2 recibe 
                                                            # parte del cromosoma del primer
                                                            # padre.
                else:
                    hijo1Cromo += cromo1[ultimoPunto:punto] # El cromosoma del hijo1 recibe 
                                                            # parte del cromosoma del primer
                                                            # padre.
                    hijo2Cromo += cromo2[ultimoPunto:punto] # El cromosoma del hijo2 recibe 
                                                            # parte del cromosoma del segundo
                                                            # padre.
                
                # Cambiamos el segmento
                switch = not switch # De esta manera garantizamos que vamos a 
                                    # ir alternando.
                ultimoPunto = punto # Nos movemos al siguiente punto de corte.
            
            # Añadimos el nuevo cromosoma a los hijos
            hijo1.append(hijo1Cromo)
            hijo2.append(hijo2Cromo)
        
        # Añadimos los dos hijos generados a la lista de hijos
        hijos.append(hijo1)
        hijos.append(hijo2)
    
    # Para este punto ya tenemos la lista de los hijos, así que lo que toca es
    # eliminar uno para hacerle espacio al individuo más apto.
    index_a_eliminar = random.randint(0, len(hijos)-1) # De manera aleatoria, 
                                                       # elegimos un índice para
                                                       # quitar un hijo de la lista.
    #print(hijos)
    #print(f'Eliminando al hijo en la posición: {index_a_eliminar}')
    del hijos[index_a_eliminar] # Borramos al hijo elegido.
    
    return hijos # Retornamos una lista de 2k-1 hijos (que era lo que queríamos)


In [10]:
'''Función que realiza la mutación de un bit elegido al azar: Por cada hijo
   elegimos un cromosoma al azar, dentro de ese cromosoma elegimos un bit al
   azar, generamos un número aleatorio r en [0,1]. Si r<probaMutar, cambiamos 
   el valor del bit, en otro caso el bit se queda igual.
   Recibimos como parámetros: la lista de los hijos (en binario) y una 
   probabilidad de mutación. La función retorna la lista (en binario) de los hijos
   que (posiblemente) han sido mutados.'''
   

import copy

def Mutador_1_flip(hijos, probaMutar):
    
    hijosMutados = []
    #huboMutacion = False # Variable para verificar si hubo al menos una mutación.

    for hijo in hijos:
        hijo = copy.deepcopy(hijo)  # Hacemos una copia profunda del hijo para no modificar el 
                                    # original. si no hacemos una copia de los hijos antes de 
                                    # realizar mutaciones, estaremos modificando las listas 
                                    # originales directamente. Lo que puede causarnos problemas
                                    # si luego intentamos usar los hijos originales en alguna 
                                    # otra operación.          

        if isinstance(hijo[0], list):  # Si el primer elemento es una lista, tenemos listas de 
                                       # cromosomas (o sea, la dimensión del problema es al 
                                       # menos 2).
            # Elegimos un cromosoma al azar dentro del hijo

            cromElegido_index = random.randint(0, len(hijo)-1) # Elegimos un índice al azar en 
                                                           # `hijo`. Este corresponde a un 
                                                           # cromosoma.
            cromElegido = hijo[cromElegido_index] # Fijamos el cromosoma elegido.

            bit_index = random.randint(0, len(cromElegido)-1) # Elegimos un índice al azar en 
                                                              # el cromosoma elegido, este 
                                                              # corresponde a alguno de los 
                                                              # nBits.
        
            r = random.random() # Generamos un número aleatorio entre 0 y 1.

            # Si el número aleatorio es menor que la probabilidad de mutación p_m
            if r < probaMutar:
            
                # Hacemos flip al bit: si es 0 lo cambiamos a 1, si es 1 lo cambiamos a 0
                cromElegido[bit_index] = 1 if cromElegido[bit_index] == 0 else 0

                #huboMutacion = True

            # Añadimos el cromosoma (posiblemente mutado) a la lista de hijosMutados
            hijosMutados.append(hijo)

        else: # Si el  no es una lista de listas, es directamente un cromosoma (o sea, la
              # dimensión del problema es 1).
        
            bit_index = random.randint(0, len(hijo)-1) # Elegimos un bit al azar dentro del 
                                                       # cromosoma (en este caso, cada hijo
                                                       # es su propio cromosoma).
            
            r = random.random() # Generamos un número aleatorio entre 0 y 1.

            # Si el número aleatorio es menor que la probabilidad de mutación p_m
            if r < probaMutar:
                hijo[bit_index] = 1 if hijo[bit_index] == 0 else 0 # Flip del bit.

                #huboMutacion = True

            # Añadimos el cromosoma (posiblemente mutado) a la lista de hijosMutados
            hijosMutados.append(hijo)
    
    #print("Hubo mutación:", huboMutacion)

    return hijosMutados


In [11]:
'''Funciones auxiliares para pasar a los hijos (posiblemente) mutados de su 
   codificación en arreglos de bits a vectores de números reales.'''

def decodifica(x_cod, nBits, intervalo):
    """
    Decodifica un vector de bits como un número real en el intervalo [a, b].

    Parámetros:
    x_cod: vector de bits a decodificar
    nBits: número de bits a utilizar
    intervalo: De la forma [a, b]

    Retorno:
    número real decodificado
    """

    a, b = intervalo
    precision = (b - a) / (2 ** nBits)  # Calcula la precisión de la representación
    indice = int(''.join(map(str, x_cod)), 2)  # Convierte el vector de bits en un número entero
    x_dec = a + indice * precision  # Calcula el número real decodificado

    return x_dec

def decodifica_vector(vector_binario, nBits, intervalo):
    """
    Decodifica un vector de arreglos binarios en un vector de números reales.

    Parámetros:
    vector_binario: lista de listas de enteros, donde cada sublista representa un número codificado en binario
    nBits: número de bits a utilizar para cada número
    a: límite inferior de la partición
    b: límite superior de la partición

    Retorno:

    lista de números reales decodificados
    """

    dim_x = len(vector_binario)


    vector_reales = []

    for i in range(dim_x):
        binario = vector_binario[i]
        # Decodifica cada número utilizando la función decodifica
        real = decodifica(binario, nBits, intervalo)
        # Añade el resultado decodificado al vector principal
        vector_reales.append(real)

    return vector_reales


In [13]:
'''Función que decodifica a los hijos mutados para el reemplazo en la 
   siguiente generación. Recibe como parámetros a loss hijos (posiblemente) 
   mutados (codificados en bits), la cantidad de nBits usada en la codificación
   y el intervalo de nuestra función objetivo.   
   Retorna la lista de los hijos como vectores de números reales.'''

def Hijos_Decodificados(hijosMutados, nBits, intervalo):
    hijosDecodificados = []
    for hijoMutado in hijosMutados:
        hijoDecodificado = decodifica_vector(hijoMutado, nBits, intervalo)
        hijosDecodificados.append(hijoDecodificado)

    return(hijosDecodificados)


In [14]:
'''Función que nos da la siguiente generacion completa. Recibe como
   parámetros la lista de los 2k-1 hijos decodificados como vectores de 
   números reales y agrega a la lista al individuo más apto para que 
   tener el reemplazo con elitismo.'''

def Siguiente_Gen(hijosDecodificados, masApto):
    siguenteGeneracion = hijosDecodificados + [masApto]
    return siguenteGeneracion


In [15]:
'''Función que realiza una ejecución del algoritmo genético. Recibe como
   parámetros: la función objetivo a minimizar, el intervalo de ésta, la
   dimensión del problema, la mitad del tamaño de la población (pues vamos
   a tomar 2*(kindividuos)), la cantidad de bits para la codificación en 
   arreglos binarios, el número de cortes para el operador de reproducción 
   de cruza, la probabilidad de mutación de un bit y el número de iteraciones
   a realizar dentro de la ejecución (el número de generaciones).'''

def Algoritmo_Genetico(funcion, intervalo, dimension,   # Cuando tienes una función que
                       kindividuos, nBits, nCortes,     # tiene muchos parámetros, la
                       probaMutar, iteraciones):        # puedes poner así para que sea
                                                        # más legible.
    
    contadorIteraciones = 0 # Contamos las iteraciones.

    # Generamos a nuestra población inicial:
    poblacion = Generar_Poblacion(kindividuos, dimension, intervalo) 
    
    while contadorIteraciones <= iteraciones:

        # Calculamos las probabilidades de selección y obtenemos el individuo más apto
        probas_pi, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion)

        # Calculamos las probabilidades acumulativas
        probasAcumulativas_qi = Probas_Acumulativas(probas_pi)

        # Seleccionamos a los padres (ponemos `_` porque no nos interesa el número de 
        # clones).
        padres, _ = Generar_Parejas(poblacion, probasAcumulativas_qi)

        # Convertimos a los padres en arreglos de bits.
        padresBinarios = Padres_Binarios(padres, nBits, intervalo)

        # Cruzamos a los padres para generar hijos (como arreglos binarios).
        hijosBinarios = Cruzar_N_Puntos(padresBinarios, nCortes, nBits)

        # Aplicamos la mutación 1-flip a los hijos.
        hijosMutados = Mutador_1_flip(hijosBinarios, probaMutar)

        # Decodificamos a los hijos a su representación original como vectores de reales.
        hijosDecodificados = Hijos_Decodificados(hijosMutados, nBits, intervalo)

        # Generamos la siguiente generación, incluyendo el individuo más apto.
        poblacion = Siguiente_Gen(hijosDecodificados, masApto)

        contadorIteraciones += 1 # Incrementamos en uno el contador de las iteraciones.

    return masApto # Devolvemos la mejor solución encontrada.


**Funciones de prueba:**

In [17]:
def Sphere(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de la esfera en el vector.
  Se usa en el intervalo de busqueda [-5.12, 5.12] y su óptimo global está en f([0,..,0])=0

  Ejemplo de uso:
    > Sphere([0,0])

    > 0
    """
  suma = 0
  for x in lista:
    suma += x**2
  return suma

intervaloSphere = (-5.12, 5.12)


def Ackley(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función Ackley en el vector.
  Se usa en el intervalo de busqueda [-30, 30] y su óptimo global está en f([0,..,0])=0

  Ejemplo de uso:
    > Ackley([0,0])

    > 0
    """
  suma1 = 0
  suma2 = 0
  n = len(lista)
  for x in lista:
    suma1 += (x**2)
    suma2 += np.cos( 2 * np.pi * x )
  return 20 + np.exp(1) - 20 * np.exp(-0.2 * np.sqrt((1/n)*suma1)) - np.exp((1/n)* suma2)

intervaloAckley = (-30, 30)


def Griewank(x):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función Griewank en el vector.
  Se usa en el intervalo de busqueda [-600, 600] y su óptimo global está en f([0,..,0])=0

  Ejemplo de uso:
    > Griewank([0,0])

    > 0
  """
  suma = 0
  m = 1 # se guardará la multiplicacion de cosenos y por eso se inicializa en 1
  n = len(x)
  for i in range(1, n+1 ):
    x_i = float(x[i-1])
    suma += ((x_i)**2) / 4000
    m = m * np.cos(x_i / np.sqrt(i))
  return 1 + suma - m

intervaloGriewank = (-600, 600)


def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  suma = 0
  for x in lista:
    suma += x**2 - 10 * np.cos(2* np.pi *x)
  return 10 * n + suma

intervaloRastrigin = (-5.12, 5.12)


def Rosenbrock(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rosenbrock en el vector.
  Tiene mínimo global en f(-2.903534, ..., -2.903534)

  Ejemplo de uso:
    > Rosenbrock([1,1])

    > 0

  """
  n = len(lista)
  suma = 0
  for i in range(n-1):
    suma += 100*(lista[i+1] - (lista[i])**2)**2 + (lista[i]-1)**2
  return suma

intervaloRosenbrock = (-2.048, 2.048)


Veamos un ejemplo de ejecución de nuestro algoritmo para nuestras funciones de prueba:

In [22]:
funciones = [(Sphere, "Sphere"), (Ackley, "Ackley"), (Griewank, "Griewank"),
             (Rastrigin, "Rastrigin"), (Rosenbrock, "Rosenbrock")]

intervalos = {"Sphere": intervaloSphere, "Ackley" : intervaloAckley,
              "Griewank": intervaloGriewank, "Rastrigin": intervaloRastrigin, 
              "Rosenbrock": intervaloRosenbrock}

dimension = 10
kindividuos = 70 # Tendremos una población del doble
nBits = 15
nCortes = 14
probaMutar = 0.45
iteraciones = 1000

for funcion, nombreFuncion in funciones:
    intervalo = intervalos[nombreFuncion]
    masApto = Algoritmo_Genetico(funcion, intervalo, dimension,   
                                 kindividuos, nBits, nCortes,     
                                 probaMutar, iteraciones)
    print(f"{nombreFuncion:<10}, {str(masApto):<10}, {funcion(masApto)}")


Sphere    , [-0.0012499999999997513, -0.0009375000000000355, 0.0, -0.0012499999999997513, 0.0, -0.0009375000000000355, 0.0003124999999997158, -0.0009375000000000355, 0.0003124999999997158, -0.0006250000000003197], 6.347656249999001e-06
Ackley    , [-0.0018310546875, -0.0018310546875, 0.0, 0.0, 0.0, 0.0, -0.0018310546875, 0.0, 0.0, 0.0], 0.0040652058544417
Griewank  , [-6.26220703125, 8.8623046875, -0.03662109375, 6.298828125, 7.03125, 0.0, 0.0, 9.375, 9.33837890625, 0.0], 0.11142858793565691
Rastrigin , [2.0, 0.9596875000000002, 1.0, 0.9596875000000002, 1.9896874999999996, -1.9900000000000002, -0.9953124999999998, 1.9196875000000002, 1.9196875000000002, -1.9900000000000002], 30.277849635742456
Rosenbrock, [0.4797500000000001, 0.20862500000000006, 0.016875000000000195, 0.00749999999999984, -0.0007500000000000284, -0.00024999999999986144, 0.008750000000000036, -0.0022500000000000853, 0.00812500000000016, 0.009749999999999925], 7.968264594672728
